In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mp
import matplotlib.animation as animation
import numpy as np
import IPython

from envs.Quadrotor import Quadrotor

In [3]:
m = 1.0
I = 1.0
r = 1.0

quadrotor = Quadrotor(m, I, r)

In [4]:
quadrotor = Quadrotor(m, I, r)
u = np.array([10, 10])

for i in range(50):
    quadrotor.step(u)

In [5]:
fig = mp.figure.Figure(figsize=[8, 8])
mp.backends.backend_agg.FigureCanvasAgg(fig)
ax = fig.add_subplot(111, autoscale_on=False, xlim=[-2, 12], ylim=[-2, 12])
ax.grid()

#create the quadrotor
center, = ax.plot([], [], 'k', marker="o")
lines = []
    
for i in range(8):
    line, = ax.plot([], [], 'k', lw=2)
    lines.append(line)

In [6]:
def drawQuadrotor(quadrotor, ax, center, lines, t):
    r = quadrotor.r
    h = r/2
    x, y, theta, u, v, omega = quadrotor.x
    
    for line in lines: #reset all lines
        line.set_data([],[])
    
    R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    t = np.array([[x], [y]])
    
    A = np.array([-r, h/2])
    B = np.array([ r, h/2])
    C = np.array([ r,-h/2])
    D = np.array([-r,-h/2])
    
    E = np.array([-r+h/2, h/2])
    F = np.array([-r+h/2, h   ])
    G = np.array([ r-h/2, h/2])
    H = np.array([ r-h/2, h   ])
    
    I = np.array([F[0]-h*np.cos(np.pi*t), F[1]])
    J = np.array([F[0]+h*np.cos(np.pi*t), F[1]])
    K = np.array([H[0]-h*np.cos(np.pi*t), H[1]])
    L = np.array([H[0]+h*np.cos(np.pi*t), H[1]])
    
    coords = np.vstack([A, B, C, D, E, F, G, H, I, J, K, L])
    coords = coords.T
    
    coords = R @ coords + t
    
    A = coords[:, 0]
    B = coords[:, 1]
    C = coords[:, 2]
    D = coords[:, 3]
    E = coords[:, 4]
    F = coords[:, 5]
    G = coords[:, 6]
    H = coords[:, 7]
    I = coords[:, 8]
    J = coords[:, 9]
    K = coords[:, 10]
    L = coords[:, 11]
    
    center.set_data([x], [y])
    
    lines[0].set_data([A[0], B[0]], [A[1], B[1]])
    lines[1].set_data([B[0], C[0]], [B[1], C[1]])
    lines[2].set_data([C[0], D[0]], [C[1], D[1]])
    lines[3].set_data([A[0], D[0]], [A[1], D[1]])
    
    lines[4].set_data([E[0], F[0]], [E[1], F[1]])
    lines[5].set_data([G[0], H[0]], [G[1], H[1]])
    
    lines[6].set_data([I[0], J[0]], [I[1], J[1]])
    lines[7].set_data([K[0], L[0]], [K[1], L[1]])
    
    return lines

In [7]:
def animate_quadrotor(quadrotor, controller, horizon):
    dt = quadrotor.dt
    
    fig = mp.figure.Figure(figsize=[12, 12])
    mp.backends.backend_agg.FigureCanvasAgg(fig)
    ax = fig.add_subplot(111, autoscale_on=False, xlim=[-2, 42], ylim=[-2, 42])
    ax.grid()

    #create the quadrotor
    center, = ax.plot([], [], 'k', marker="o")
    lines = []

    for i in range(8):
        line, = ax.plot([], [], 'k', lw=2)
        lines.append(line)
    
    # simulate with controller
    def animate(i):
        nonlocal lines
        t = dt * i
        lines = drawQuadrotor(quadrotor, ax, center, lines, t)
        u = controller(quadrotor.x)
        quadrotor.step(u)
        return lines
        
    def init():
        return animate(0)
    
    ani = animation.FuncAnimation(fig, animate, np.arange(0, horizon),
        interval=1000*dt, blit=True, init_func=init)
    plt.close(fig)
    plt.close(ani._fig)
    IPython.display.display_html(IPython.core.display.HTML(ani.to_html5_video()))

In [8]:
def controller(x):
    return np.random.randint(0, 20, 2)

In [9]:
quadrotor = Quadrotor(m, I, r)

# animate_quadrotor(quadrotor, lambda x: np.array([5, 5]), 1000)

# Debug on Buffer

In [10]:
from infrastructure.buffer import ReplayBuffer

In [11]:
buffer = ReplayBuffer()

quadrotor = Quadrotor(m, I, r)

x = quadrotor.reset()
print(x)
terminal = False
i = 0

while not terminal:
    u = controller(x)
    x_next, cost, terminal = quadrotor.step(u)
    buffer.push((x, u, cost, x_next, terminal))
    
    x = x_next
    i += 1

print(f"Simulation ends in {i} steps")

[0. 0. 0. 0. 0. 0.]
Simulation ends in 67 steps


# Debug on Agent

In [21]:
from utils.util import build_network
from agents.pg_agent import PGAgent

import tensorflow as tf

In [22]:
n_states = 6
n_actions = 2

q_network = build_network(n_states+n_actions, 1, 4, 512)
p_network = build_network(n_states, n_actions, 4, 512, "relu")

pg = PGAgent(q_network, p_network, 0.99, 0.1, 1e-5, 1e-5)

# animate_quadrotor(quadrotor, pg, 1000)

In [23]:
# tf.keras.backend.set_floatx('float64')

quadrotor = Quadrotor(m, I, r)
buffer = ReplayBuffer(10**4)

print("Start pre-sampling")
for t in range(10):
    x = quadrotor.reset()
    terminal = False
    i = 0
    
    while not terminal:
        u = np.random.rand(2) * 20
        x_next, cost, terminal = quadrotor.step(u)
        buffer.push((x, u, cost, x_next, terminal))
        x = x_next
        
        i += 1
    
    print(f"Simulation ends in {i} steps")

for t in range(1000):
    print(f"Episode {t} begins...")
    x = quadrotor.reset()
    terminal = False
    i = 0
    
    mu = 10*np.exp(- t/100)
    std = np.exp(- t/100)
    
    while not terminal:
        u = pg(x)
#         u = u[0]
        noise = np.random.normal(loc=mu, scale=std, size=2)
        print(f"Current x: {(x[0], x[1], x[2])}, u: {u}, noise: {noise}")
        u += noise
        u[u<0] = 0
        
        x_next, cost, terminal = quadrotor.step(u)
        buffer.push((x, u, cost, x_next, terminal))
    
        x = x_next
        i+=1
    
#     if t % 10 == 0:
        print("Start training ...")
        exps = buffer.sample(64)
        L, J = pg.train(exps)

#         pg.update_target_networks()
        
        print(f"Step: {t}, L: {L}, J: {J}")
    
    print(f"Simulation ends in {i} steps")


Start pre-sampling
Simulation ends in 60 steps
Simulation ends in 67 steps
Simulation ends in 62 steps
Simulation ends in 64 steps
Simulation ends in 65 steps
Simulation ends in 63 steps
Simulation ends in 2 steps
Simulation ends in 62 steps
Simulation ends in 59 steps
Simulation ends in 68 steps
Episode 0 begins...

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Current x: (0.0, 0.0, 0.0), u: [0. 0.], noise: [10.57494488  8.22049028]
Start training ...
Step: 0, L: 642.1651611328125, J: 0.2938149571418762
Current x: (0.0, 0.0, 0.0), u: [0.         0.

Step: 0, L: 407.9801330566406, J: 0.6954777836799622
Current x: (-0.0002690408264607418, 0.7680114199828404, 0.0007175097465515139), u: [0. 0.], noise: [8.83360835 9.17099813]
Start training ...
Step: 0, L: 506.9350891113281, J: 0.7314584851264954
Current x: (-0.0003175597409689042, 0.808842933178479, 0.0002497191429138186), u: [0. 0.], noise: [11.66107784 10.0305722 ]
Start training ...
Step: 0, L: 288.8062744140625, J: 0.71497642993927
Current x: (-0.00036737050354957655, 0.8504939067260905, -0.00025181045532226545), u: [0. 0.], noise: [10.54619557 10.90371574]
Start training ...
Step: 0, L: 426.2203369140625, J: 0.7074615955352783
Current x: (-0.00041772294815901233, 0.8933330452451302, -0.0005902894973754882), u: [0. 0.], noise: [ 9.50198995 10.02636266]
Start training ...
Step: 0, L: 410.57196044921875, J: 0.8405919075012207
Current x: (-0.0004675352615376139, 0.937336174998655, -0.0009645205497741699), u: [0. 0.], noise: [9.06586785 9.35451731]
Start training ...
Step: 0, L: 437.

Current x: (0.00043293004339365514, 0.31177877189218345, -0.005335911273956298), u: [0. 0.], noise: [ 9.59352368 10.6691649 ]
Start training ...
Step: 1, L: 313.1889953613281, J: 1.3915596008300781
Current x: (0.0005023088933286717, 0.3374661579302064, -0.006026474666595458), u: [0. 0.], noise: [ 9.39514587 10.58962443]
Start training ...
Step: 1, L: 289.6264343261719, J: 1.4348163604736328
Current x: (0.0005824996823222485, 0.3641987838906977, -0.006824602127075195), u: [0. 0.], noise: [10.11702703  8.93750694]
Start training ...
Step: 1, L: 203.83596801757812, J: 1.2793500423431396
Current x: (0.0006747341693187161, 0.3919488505426754, -0.007742177486419677), u: [0. 0.], noise: [9.2882466  9.04716221]
Start training ...
Step: 1, L: 420.8195495605469, J: 1.5973851680755615
Current x: (0.0007799725173209043, 0.4206233263126031, -0.008541800785064698), u: [0. 0.], noise: [8.91920161 9.39660719]
Start training ...
Step: 1, L: 303.7427062988281, J: 1.574553370475769
Current x: (0.00089940

Step: 2, L: 280.83612060546875, J: 2.2368216514587402
Current x: (-1.6776238108504014e-06, 0.026074621599607772, -0.0006735400199890137), u: [0. 0.], noise: [8.37385519 9.82319718]
Start training ...
Step: 2, L: 296.3912048339844, J: 2.1617770195007324
Current x: (-8.534406591169604e-07, 0.033617849441245226, -0.001105440616607666), u: [0. 0.], noise: [9.69078601 9.48661945]
Start training ...
Step: 2, L: 390.6058349609375, J: 2.2774088382720947
Current x: (1.1963866768599668e-06, 0.04199978207031778, -0.0016822753906250002), u: [0. 0.], noise: [ 9.33019308 11.78926558]
Start training ...
Step: 2, L: 359.12335205078125, J: 2.428126335144043
Current x: (5.366161966815112e-06, 0.05131845415875544, -0.0022386934280395512), u: [0. 0.], noise: [10.35670354 12.01836588]
Start training ...
Step: 2, L: 224.72654724121094, J: 2.149566411972046
Current x: (1.3088810220477306e-05, 0.06176806917395919, -0.003041018676757813), u: [0. 0.], noise: [9.05290974 8.25242723]
Start training ...
Step: 2, L

Step: 2, L: 261.2224426269531, J: 3.112239360809326
Current x: (0.029434987335729856, 1.5715225792783318, -0.03987240934371948), u: [0. 0.], noise: [10.62419285  9.70346888]
Start training ...
Step: 2, L: 219.420654296875, J: 3.3224120140075684
Current x: (0.03131376569917156, 1.6285018035299845, -0.04060716824531555), u: [0. 0.], noise: [ 9.398101  10.8057466]
Start training ...
Step: 2, L: 225.44529724121094, J: 3.1555867195129395
Current x: (0.033273573880002676, 1.686531178482724, -0.04124985475540161), u: [0. 0.], noise: [9.47829269 8.297595  ]
Start training ...
Step: 2, L: 174.22293090820312, J: 3.1777987480163574
Current x: (0.035315401620606424, 1.7455982727040766, -0.04203330588340759), u: [0. 0.], noise: [10.02714324 10.42879276]
Start training ...
Step: 2, L: 207.46002197265625, J: 3.36088228225708
Current x: (0.0374305338423107, 1.805460443464977, -0.04269868726730346), u: [0. 0.], noise: [10.87720932  8.10474209]
Start training ...
Step: 2, L: 224.24319458007812, J: 3.279

...

Step: 893, L: 432.5190124511719, J: -44.76177978515625
Simulation ends in 2 steps
Episode 894 begins...
Current x: (0.0, 0.0, 0.0), u: [3.5161774 2.7192547], noise: [0.00121224 0.00144225]
Start training ...
Step: 894, L: 227.15830993652344, J: -46.94317626953125
Current x: (0.0, 0.0, 0.0), u: [2.0515335 1.885911 ], noise: [0.00140011 0.00115151]
Start training ...
Step: 894, L: 202.69467163085938, J: -46.597755432128906
Simulation ends in 2 steps
Episode 895 begins...
Current x: (0.0, 0.0, 0.0), u: [3.4699044 2.7217815], noise: [0.00121593 0.00140119]
Start training ...
Step: 895, L: 195.38723754882812, J: -43.99102783203125
Current x: (0.0, 0.0, 0.0), u: [2.0028164 1.8730289], noise: [0.00128728 0.00115659]
Start training ...
Step: 895, L: 292.75433349609375, J: -44.36170196533203
Simulation ends in 2 steps
Episode 896 begins...
Current x: (0.0, 0.0, 0.0), u: [3.4244244 2.7242134], noise: [0.00133423 0.00117288]
Start training ...
Step: 896, L: 347.6348571777344, J: -44.7772674560546

Step: 917, L: 297.7806091308594, J: -49.704158782958984
Current x: (0.0, 0.0, 0.0), u: [1.4990156 1.8408607], noise: [0.00105731 0.00121657]
Start training ...
Step: 917, L: 126.47036743164062, J: -49.926734924316406
Simulation ends in 2 steps
Episode 918 begins...
Current x: (0.0, 0.0, 0.0), u: [2.8523455 2.8848329], noise: [0.00093147 0.00108987]
Start training ...
Step: 918, L: 63.13903045654297, J: -51.56139373779297
Current x: (0.0, 0.0, 0.0), u: [1.4833876 1.8409411], noise: [0.00092162 0.00096394]
Start training ...
Step: 918, L: 472.91864013671875, J: -50.128787994384766
Simulation ends in 2 steps
Episode 919 begins...
Current x: (0.0, 0.0, 0.0), u: [2.8354282 2.8935976], noise: [0.00104004 0.00106997]
Start training ...
Step: 919, L: 64.30701446533203, J: -52.115997314453125
Current x: (0.0, 0.0, 0.0), u: [1.4725299 1.8446856], noise: [0.00095438 0.00115601]
Start training ...
Step: 919, L: 132.7529754638672, J: -51.655704498291016
Simulation ends in 2 steps
Episode 920 begins

Step: 942, L: 70.76934051513672, J: -53.70798873901367
Simulation ends in 2 steps
Episode 943 begins...
Current x: (0.0, 0.0, 0.0), u: [2.4046533 3.0097368], noise: [0.00075838 0.00088197]
Start training ...
Step: 943, L: 140.5048370361328, J: -53.20489501953125
Current x: (0.0, 0.0, 0.0), u: [1.1240324 1.806983 ], noise: [0.00076056 0.0007918 ]
Start training ...
Step: 943, L: 23.175256729125977, J: -53.81023406982422
Simulation ends in 2 steps
Episode 944 begins...
Current x: (0.0, 0.0, 0.0), u: [2.3909032 3.0164666], noise: [0.00081446 0.0006996 ]
Start training ...
Step: 944, L: 253.84500122070312, J: -53.505428314208984
Current x: (0.0, 0.0, 0.0), u: [1.1174115 1.8107734], noise: [0.00101446 0.0008313 ]
Start training ...
Step: 944, L: 80.64859008789062, J: -53.69232177734375
Simulation ends in 2 steps
Episode 945 begins...
Current x: (0.0, 0.0, 0.0), u: [2.3798022 3.0249987], noise: [0.00085486 0.00073404]
Start training ...
Step: 945, L: 15.07886028289795, J: -54.77546691894531


Step: 966, L: 78.28267669677734, J: -54.17691421508789
Simulation ends in 2 steps
Episode 967 begins...
Current x: (0.0, 0.0, 0.0), u: [2.1305602 3.184262 ], noise: [0.0006414  0.00068147]
Start training ...
Step: 967, L: 79.03255462646484, J: -54.46583557128906
Current x: (0.0, 0.0, 0.0), u: [0.9755693 1.896898 ], noise: [0.00064597 0.00075451]
Start training ...
Step: 967, L: 178.0564727783203, J: -54.026824951171875
Simulation ends in 2 steps
Episode 968 begins...
Current x: (0.0, 0.0, 0.0), u: [2.1176174 3.189056 ], noise: [0.00060805 0.00061271]
Start training ...
Step: 968, L: 152.6244354248047, J: -54.50937271118164
Current x: (0.0, 0.0, 0.0), u: [0.9658774 1.8963194], noise: [0.00058533 0.00057379]
Start training ...
Step: 968, L: 389.2027587890625, J: -52.842472076416016
Simulation ends in 2 steps
Episode 969 begins...
Current x: (0.0, 0.0, 0.0), u: [2.1028087 3.1917055], noise: [0.00061373 0.00071999]
Start training ...
Step: 969, L: 237.07772827148438, J: -54.88654327392578


Step: 991, L: 80.33447265625, J: -55.38845443725586
Simulation ends in 2 steps
Episode 992 begins...
Current x: (0.0, 0.0, 0.0), u: [1.8348821 3.27791  ], noise: [0.00053943 0.00055049]
Start training ...
Step: 992, L: 53.58854675292969, J: -55.44139099121094
Current x: (0.0, 0.0, 0.0), u: [0.75936973 1.8822845 ], noise: [0.00053224 0.00056192]
Start training ...
Step: 992, L: 193.19668579101562, J: -55.08283996582031
Simulation ends in 2 steps
Episode 993 begins...
Current x: (0.0, 0.0, 0.0), u: [1.8266143 3.2833114], noise: [0.00045881 0.0004906 ]
Start training ...
Step: 993, L: 109.60499572753906, J: -55.13428497314453
Current x: (0.0, 0.0, 0.0), u: [0.75431037 1.8845406 ], noise: [0.00052378 0.0005338 ]
Start training ...
Step: 993, L: 186.90219116210938, J: -55.312095642089844
Simulation ends in 2 steps
Episode 994 begins...
Current x: (0.0, 0.0, 0.0), u: [1.8178244 3.2878265], noise: [0.00047236 0.00049596]
Start training ...
Step: 994, L: 125.84571838378906, J: -55.581089019775

In [15]:
xs, us, cs, x_nexts = [], [], [], []
for exp in exps:
    x, u, c, x_next, _ = exp
    xs.append(x)
    us.append(u)
    cs.append(c)
    x_nexts.append(x_next)

            # q = self.q_network.forward(x, u)
            # a = self.target_p_network.forward(x_next)
            # q_= self.target_q_network.forward(x_next, a)

            # l = (q - c - self.alpha * q_)**2
            # j = self.q_network.forward(x, self.p_network(x))

            # L += l
            # J += j
        
        # xs = torch.as_tensor(np.array(xs)).float()
        # us = torch.as_tensor(np.array(us)).float()
        # cs = torch.as_tensor(np.array(cs)).float()
        # x_nexts = torch.as_tensor(np.array(x_nexts)).float()

        
xs = tf.convert_to_tensor(np.array(xs), dtype=tf.float32)
us = tf.convert_to_tensor(np.array(us), dtype=tf.float32)
cs = tf.convert_to_tensor(np.array(cs), dtype=tf.float32)
x_nexts = tf.convert_to_tensor(np.array(x_nexts), dtype=tf.float32)

In [16]:
u_nexts = pg.target_p_network(x_nexts)
# u_nexts = tf.cast(u_nexts, tf.float32)
u_nexts

<tf.Tensor: shape=(64, 2), dtype=float32, numpy=
array([[-27.397186 ,  -3.6126623],
       [-20.294073 ,  -3.0222569],
       [-19.01974  ,  -2.848195 ],
       [-19.38033  ,  -2.8981712],
       [-19.01974  ,  -2.848195 ],
       [-19.01974  ,  -2.848195 ],
       [-28.191956 ,  -3.3507142],
       [-26.69639  ,  -3.7980733],
       [-26.883615 ,  -3.5038812],
       [-17.440256 ,  -2.660512 ],
       [-20.086332 ,  -2.9943938],
       [-27.932484 ,  -3.5136137],
       [-16.571728 ,  -2.5351715],
       [-19.829245 ,  -2.9596193],
       [-19.01974  ,  -2.848195 ],
       [-27.993149 ,  -3.5337703],
       [-28.11476  ,  -3.4955015],
       [-17.214678 ,  -2.6281576],
       [-20.005571 ,  -2.9835043],
       [-22.90214  ,  -3.299993 ],
       [-24.485182 ,  -3.5085058],
       [-15.993697 ,  -2.4506803],
       [-17.194021 ,  -2.6251884],
       [-28.204874 ,  -3.355771 ],
       [-28.206867 ,  -3.3618321],
       [-28.20878  ,  -3.3570218],
       [-19.01974  ,  -2.848195 ],
      

In [17]:
tf.concat([x_nexts, u_nexts], 1)

<tf.Tensor: shape=(64, 8), dtype=float32, numpy=
array([[ 3.25826253e-03,  1.19594403e-01, -6.22657947e-02,
         4.96867262e-02,  4.21463728e-01, -4.84144479e-01,
        -2.73971863e+01, -3.61266232e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -6.76920637e-02, -3.04079335e-02,
        -2.02940731e+01, -3.02225685e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -9.80999991e-02,  0.00000000e+00,
        -1.90197392e+01, -2.84819508e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -8.99247900e-02, -8.17520823e-03,
        -1.93803291e+01, -2.89817119e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -9.80999991e-02,  0.00000000e+00,
        -1.90197392e+01, -2.84819508e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -9.80999991e-02,  0.00000000e+00,
        -1.90197392e+01, -2.

In [18]:
x_nexts

<tf.Tensor: shape=(64, 6), dtype=float32, numpy=
array([[ 3.25826253e-03,  1.19594403e-01, -6.22657947e-02,
         4.96867262e-02,  4.21463728e-01, -4.84144479e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -6.76920637e-02, -3.04079335e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -9.80999991e-02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -8.99247900e-02, -8.17520823e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -9.80999991e-02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -9.80999991e-02,  0.00000000e+00],
       [-2.83988323e-02,  8.16333771e-01,  7.32125193e-02,
        -2.45994374e-01,  3.85994387e+00,  1.96686253e-01],
       [ 1.35652733e-03,  4.46521072e-03, -8.00625831e-02,
         3.17685790e-02, -1.44398987e-01, -1.02543187e+00],

In [19]:
u

array([2.9685502, 6.2623725], dtype=float32)

In [20]:
quadrotor.reset()
animate_quadrotor(quadrotor, pg, 1000)

CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '864x864', '-pix_fmt', 'rgba', '-r', '100.0', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-y', '/tmp/tmpl1gtbdgx/temp.m4v']' returned non-zero exit status 255.

In [ ]:
eps

In [ ]:
x = exps[0][0]
x_next = exps[0][3]

In [ ]:
p_network(torch.as_tensor(x).float())

In [ ]:
torch.as_tensor(x).float()

In [ ]:
exps

In [ ]:
pg(x)[0]

In [ ]:
for _ in range(1000):
    if np.random.rand() < eps:
        u = np.random.rand(2) * 10 + 10

        print(u)

In [ ]:
u = pg(x)
u

In [ ]:
len(x.shape)

In [ ]:
q_network.trainable_weights

In [ ]:
q_network